# Setting

In [ ]:
# 코드나 내용에서 변경있으면 바로 업데이트해서 사용할 수 있게 하는 것
%load_ext autoreload
%autoreload 2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive')

Mounted at /content/drive


In [ ]:
# 현재 활동하고 있는 경로 
os.getcwd()

'/content/drive/MyDrive'

In [ ]:
# 간혹 경로문제가 생길까봐 검색할 때 사용할 기본 경로
# sys.path 는 디렉터리의 경로들이 기록된 문자열 리스트
#이 리스트에 경로를 추가하면 해당 경로에 있는 파이썬 파일을 import 문으로 불러올 수 있음
import sys

sys.path.append('/content/drive/MyDrive')

In [ ]:
sys.path

['/content',
 '/env/python',
 '/usr/lib/python37.zip',
 '/usr/lib/python3.7',
 '/usr/lib/python3.7/lib-dynload',
 '',
 '/usr/local/lib/python3.7/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.7/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/content/drive/MyDrive']

In [ ]:
# 우리가 사용하는 GPU 상태
!nvidia-smi

Thu Oct  6 08:17:16 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 16.5 MB/s 
     |████████████████████████████████| 6.6 MB 61.3 MB/s 
     |████████████████████████████████| 163 kB 74.2 MB/s 


# Package


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader

import json
import yaml
import random
import pickle
import numpy as np
from tqdm import tqdm
import datetime


In [ ]:
# from utils.data.kogpt_handler import KoGPTDataset

# Task...

## Load Data

In [ ]:
# with open("db/scripts/신세계.pkl", 'rb') as f:
#   raw = pickle.load(f)
# raw[:10]

In [ ]:
# raw = list(map(lambda x: x.strip(' '), raw))
# raw = list(map(lambda x: x.strip('.'), raw))
# # raw = list(map(lambda x: x.strip(''), raw))
# raw = [i for i in raw if i != '']
# raw[:10]

In [ ]:
# with open("style_linesci_auto.pkl", 'rb') as f:
with open("criminal.txt", 'r') as f:
  raw = f.readlines()
raw_two = raw[0].split('   ')
raw_two = list(map(lambda x: x[:-3], raw_two))
raw_two = [i for i in raw_two if len(i)>3]
# raw.keys()

In [ ]:
# for k,v in raw.items():
#   print(k, len(v))


"코미디" : 0,
"드라마": 1,
"액션" : 2,
"범죄" : 2,
"멜로/로맨스": 3

In [ ]:
# 쓰고 싶은 대본 종류 정하기
# raw_two = raw['액션'] + raw['범죄']
# raw_two = raw['코미디']
len(raw_two)

1255

In [ ]:
raw_two[:100]

## Tokenizer

In [ ]:
# !pip uninstall requests
!pip install requests
!pip install urllib3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 29.1 MB/s 


In [ ]:
# coding=utf-8
# Copyright 2018 Google AI, Google Brain and Carnegie Mellon University Authors and the HuggingFace Inc. team and Jangwon Park
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
""" Tokenization classes for KoBERT model """


import logging
import os
import unicodedata
from shutil import copyfile

from transformers import PreTrainedTokenizer

logger = logging.getLogger(__name__)

VOCAB_FILES_NAMES = {
    "vocab_file": "tokenizer_78b3253a26.model",
    "vocab_txt": "vocab.txt",
}

PRETRAINED_VOCAB_FILES_MAP = {
    "vocab_file": {
        "monologg/kobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert/tokenizer_78b3253a26.model",
        "monologg/kobert-lm": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert-lm/tokenizer_78b3253a26.model",
        "monologg/distilkobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/distilkobert/tokenizer_78b3253a26.model",
    },
    "vocab_txt": {
        "monologg/kobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert/vocab.txt",
        "monologg/kobert-lm": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert-lm/vocab.txt",
        "monologg/distilkobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/distilkobert/vocab.txt",
    },
}

PRETRAINED_POSITIONAL_EMBEDDINGS_SIZES = {
    "monologg/kobert": 512,
    "monologg/kobert-lm": 512,
    "monologg/distilkobert": 512,
}

PRETRAINED_INIT_CONFIGURATION = {
    "monologg/kobert": {"do_lower_case": False},
    "monologg/kobert-lm": {"do_lower_case": False},
    "monologg/distilkobert": {"do_lower_case": False},
}

SPIECE_UNDERLINE = "▁"

# 토큰나이저 클래스
class KoBertTokenizer(PreTrainedTokenizer):
    """
    SentencePiece based tokenizer. Peculiarities:
        - requires `SentencePiece <https://github.com/google/sentencepiece>`_
    """

    vocab_files_names = VOCAB_FILES_NAMES
    pretrained_vocab_files_map = PRETRAINED_VOCAB_FILES_MAP
    pretrained_init_configuration = PRETRAINED_INIT_CONFIGURATION
    max_model_input_sizes = PRETRAINED_POSITIONAL_EMBEDDINGS_SIZES

    def __init__(
        self,
        vocab_file,
        vocab_txt,
        do_lower_case=False,
        remove_space=True,
        keep_accents=False,
        unk_token="[UNK]",
        sep_token="[SEP]",
        pad_token="[PAD]",
        cls_token="[CLS]",
        mask_token="[MASK]",
        **kwargs,
    ):
        super().__init__(
            unk_token=unk_token,
            sep_token=sep_token,
            pad_token=pad_token,
            cls_token=cls_token,
            mask_token=mask_token,
            **kwargs,
        )

        # Build vocab
        self.token2idx = dict()
        self.idx2token = []
        with open(vocab_txt, "r", encoding="utf-8") as f:
            for idx, token in enumerate(f):
                token = token.strip()
                self.token2idx[token] = idx
                self.idx2token.append(token)

        try:
            import sentencepiece as spm
        except ImportError:
            logger.warning(
                "You need to install SentencePiece to use KoBertTokenizer: https://github.com/google/sentencepiece"
                "pip install sentencepiece"
            )

        self.do_lower_case = do_lower_case
        self.remove_space = remove_space
        self.keep_accents = keep_accents
        self.vocab_file = vocab_file
        self.vocab_txt = vocab_txt

        self.sp_model = spm.SentencePieceProcessor()
        self.sp_model.Load(vocab_file)

    @property
    def vocab_size(self):
        return len(self.idx2token)

    def get_vocab(self):
        return dict(self.token2idx, **self.added_tokens_encoder)

    def __getstate__(self):
        state = self.__dict__.copy()
        state["sp_model"] = None
        return state

    def __setstate__(self, d):
        self.__dict__ = d
        try:
            import sentencepiece as spm
        except ImportError:
            logger.warning(
                "You need to install SentencePiece to use KoBertTokenizer: https://github.com/google/sentencepiece"
                "pip install sentencepiece"
            )
        self.sp_model = spm.SentencePieceProcessor()
        self.sp_model.Load(self.vocab_file)

    def preprocess_text(self, inputs):
        if self.remove_space:
            outputs = " ".join(inputs.strip().split())
        else:
            outputs = inputs
        outputs = outputs.replace("``", '"').replace("''", '"')

        if not self.keep_accents:
            outputs = unicodedata.normalize("NFKD", outputs)
            outputs = "".join([c for c in outputs if not unicodedata.combining(c)])
        if self.do_lower_case:
            outputs = outputs.lower()

        return outputs

    def _tokenize(self, text):
        """Tokenize a string."""
        text = self.preprocess_text(text)
        pieces = self.sp_model.encode(text, out_type=str)
        new_pieces = []
        for piece in pieces:
            if len(piece) > 1 and piece[-1] == str(",") and piece[-2].isdigit():
                cur_pieces = self.sp_model.EncodeAsPieces(piece[:-1].replace(SPIECE_UNDERLINE, ""))
                if piece[0] != SPIECE_UNDERLINE and cur_pieces[0][0] == SPIECE_UNDERLINE:
                    if len(cur_pieces[0]) == 1:
                        cur_pieces = cur_pieces[1:]
                    else:
                        cur_pieces[0] = cur_pieces[0][1:]
                cur_pieces.append(piece[-1])
                new_pieces.extend(cur_pieces)
            else:
                new_pieces.append(piece)

        return new_pieces

    def _convert_token_to_id(self, token):
        """ Converts a token (str/unicode) in an id using the vocab. """
        return self.token2idx.get(token, self.token2idx[self.unk_token])

    def _convert_id_to_token(self, index):
        """Converts an index (integer) in a token (string/unicode) using the vocab."""
        return self.idx2token[index]

    def convert_tokens_to_string(self, tokens):
        """Converts a sequence of tokens (strings for sub-words) in a single string."""
        out_string = "".join(tokens).replace(SPIECE_UNDERLINE, " ").strip()
        return out_string

    def build_inputs_with_special_tokens(self, token_ids_0, token_ids_1=None):
        """
        Build model inputs from a sequence or a pair of sequence for sequence classification tasks
        by concatenating and adding special tokens.
        A KoBERT sequence has the following format:
            single sequence: [CLS] X [SEP]
            pair of sequences: [CLS] A [SEP] B [SEP]
        """
        if token_ids_1 is None:
            return [self.cls_token_id] + token_ids_0 + [self.sep_token_id]
        cls = [self.cls_token_id]
        sep = [self.sep_token_id]
        return cls + token_ids_0 + sep + token_ids_1 + sep

    def get_special_tokens_mask(self, token_ids_0, token_ids_1=None, already_has_special_tokens=False):
        """
        Retrieves sequence ids from a token list that has no special tokens added. This method is called when adding
        special tokens using the tokenizer ``prepare_for_model`` or ``encode_plus`` methods.
        Args:
            token_ids_0: list of ids (must not contain special tokens)
            token_ids_1: Optional list of ids (must not contain special tokens), necessary when fetching sequence ids
                for sequence pairs
            already_has_special_tokens: (default False) Set to True if the token list is already formated with
                special tokens for the model
        Returns:
            A list of integers in the range [0, 1]: 0 for a special token, 1 for a sequence token.
        """

        if already_has_special_tokens:
            if token_ids_1 is not None:
                raise ValueError(
                    "You should not supply a second sequence if the provided sequence of "
                    "ids is already formated with special tokens for the model."
                )
            return list(
                map(
                    lambda x: 1 if x in [self.sep_token_id, self.cls_token_id] else 0,
                    token_ids_0,
                )
            )

        if token_ids_1 is not None:
            return [1] + ([0] * len(token_ids_0)) + [1] + ([0] * len(token_ids_1)) + [1]
        return [1] + ([0] * len(token_ids_0)) + [1]

    def create_token_type_ids_from_sequences(self, token_ids_0, token_ids_1=None):
        """
        Creates a mask from the two sequences passed to be used in a sequence-pair classification task.
        A KoBERT sequence pair mask has the following format:
        0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
        | first sequence    | second sequence
        if token_ids_1 is None, only returns the first portion of the mask (0's).
        """
        sep = [self.sep_token_id]
        cls = [self.cls_token_id]
        if token_ids_1 is None:
            return len(cls + token_ids_0 + sep) * [0]
        return len(cls + token_ids_0 + sep) * [0] + len(token_ids_1 + sep) * [1]

    def save_vocabulary(self, save_directory, filename_prefix):
        """Save the sentencepiece vocabulary (copy original file) and special tokens file
        to a directory.
        """
        if not os.path.isdir(save_directory):
            logger.error("Vocabulary path ({}) should be a directory".format(save_directory))
            return

        # 1. Save sentencepiece model
        out_vocab_model = os.path.join(save_directory, VOCAB_FILES_NAMES["vocab_file"])

        if os.path.abspath(self.vocab_file) != os.path.abspath(out_vocab_model):
            copyfile(self.vocab_file, out_vocab_model)

        # 2. Save vocab.txt
        index = 0
        out_vocab_txt = os.path.join(save_directory, VOCAB_FILES_NAMES["vocab_txt"])
        with open(out_vocab_txt, "w", encoding="utf-8") as writer:
            for token, token_index in sorted(self.token2idx.items(), key=lambda kv: kv[1]):
                if index != token_index:
                    logger.warning(
                        "Saving vocabulary to {}: vocabulary indices are not consecutive."
                        " Please check that the vocabulary is not corrupted!".format(out_vocab_txt)
                    )
                    index = token_index
                writer.write(token + "\n")
                index += 1

        return out_vocab_model, out_vocab_txt

In [ ]:
from transformers import PreTrainedTokenizerFast
from transformers import PreTrainedTokenizer

from transformers import AutoModel, AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

Downloading:   0%|          | 0.00/289 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/425 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/495k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
# tokenizer = PreTrainedTokenizerFast.from_pretrained("klue/bert-base", 
#                                                     bos_token='</s>', 
#                                                     eos_token='</s>', 
#                                                     unk_token='<unk>',
#                                                     pad_token='<pad>', 
#                                                     mask_token='<mask>')

In [ ]:
for k, v in tokenizer.vocab.items():
  if v==1:
    print(k, v)

[UNK] 1


In [ ]:
idx = 8
print("Tokenize 전 : ", raw_two[idx])
print("Tokenize 후 : ", tokenizer.tokenize(raw_two[idx]))
print("Tokenize 후 : ", tokenizer.encode(raw_two[idx]))

Tokenize 전 :   사람들이 왜 도박을 하냐구요? 글쎄요.. 대부분 스릴 때문에 한다고 하잖아요? 따느냐 잃느냐. 근데 정말 그럴까요? 어느 날 고니가 얘기해 주더라고요. 사람들이 도박을 왜 하는지
Tokenize 후 :  ['사람', '##들이', '왜', '도박', '##을', '하', '##냐', '##구요', '?', '글쎄요', '.', '.', '대부분', '스릴', '때문', '##에', '한다고', '하', '##잖아요', '?', '따', '##느', '##냐', '잃', '##느', '##냐', '.', '근데', '정말', '그럴까요', '?', '어느', '날', '고', '##니', '##가', '얘기', '##해', '주', '##더라', '##고', '##요', '.', '사람', '##들이', '도박', '##을', '왜', '하', '##는지']
Tokenize 후 :  [2, 3611, 7285, 1460, 8902, 2069, 1889, 2529, 5515, 35, 15817, 18, 18, 4057, 15314, 3624, 2170, 4034, 1889, 20754, 35, 897, 2922, 2529, 1509, 2922, 2529, 18, 4224, 3944, 31255, 35, 3875, 721, 594, 2209, 2116, 3880, 2097, 1564, 23677, 2088, 2182, 18, 3611, 7285, 8902, 2069, 1460, 1889, 18246, 3]


## Model

In [ ]:
from transformers import EncoderDecoderModel
bert2bert = EncoderDecoderModel.from_encoder_decoder_pretrained("klue/bert-base", 
                                                                "klue/bert-base")

Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at klue/bert-base were not used when initializing BertLMHeadModel: ['cls.seq_relationship

## Dataset

In [ ]:
# 이걸 맨 마지막에 해야함
# 
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 432 kB 18.8 MB/s 
     |████████████████████████████████| 212 kB 67.7 MB/s 
     |████████████████████████████████| 115 kB 67.4 MB/s 
     |████████████████████████████████| 127 kB 72.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
import numpy as np

[autoreload of urllib3.connectionpool failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/usr/local/lib/python3.7/dist-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/usr/lib/python3.7/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/usr/lib/python3.7/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 630, in _exec
  File "<frozen importlib._bootstrap_external>", line 728, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py", line 46, in <module>
    from .util.url import (
ImportError: cannot import name '_normalize_host' from 'urllib3.util.url' (/usr/local/lib/python3.7/dist

In [ ]:
from datasets import Dataset

In [ ]:
raw_ds = Dataset.from_dict({"text":raw_two# list
                            
                            })

In [ ]:
enc_max_len = 32
dec_max_len = 32

def data_to_input(batch):
    # tokenize the inputs and labels
    inputs = tokenizer(batch["text"], padding="max_length", truncation=True, max_length=enc_max_len)
    # [나 , 밥, -을, 먹-,었-,다]
    # 노이즈 예씨
    # [나 , MASK, -을, 먹-,었-,다]
    # [나 , 치킨, -을, 먹-,었-,다]
    # (in)문장 --> noise --> 문장(out)
    
    # StyleLM
    # Noise
    p_drop = 0.15*0.8
    idx = np.random.choice([0, 1], size= len(inputs), p=[p_drop, 1-p_drop])
    for i in idx:
      if i == 0:
        inputs[i] = 32003 # 32003번은 MASK 토큰의 ID --> BLANK for klue

    outputs = tokenizer(batch["text"], padding="max_length", truncation=True, max_length=dec_max_len)

    batch["input_ids"] = [i[1:] for i in inputs.input_ids]
    batch["attention_mask"] = [i[1:] for i in inputs.attention_mask]
    batch["decoder_input_ids"] = [i[1:] for i in inputs.input_ids]
    batch["decoder_attention_mask"] = [i[1:] for i in outputs.attention_mask]
    batch["labels"] = [i[1:] for i in outputs.input_ids]
#     batch["input_ids"] = torch.LongTensor(inputs.input_ids).to("cuda")
#     batch["attention_mask"] = torch.LongTensor(inputs.attention_mask).to("cuda")
#     batch["decoder_input_ids"] = torch.LongTensor(outputs.input_ids).to("cuda")
#     batch["decoder_attention_mask"] = torch.LongTensor(outputs.attention_mask).to("cuda")
#     batch["labels"] = torch.LongTensor(outputs.input_ids).to("cuda")

    # because BERT automatically shifts the labels, the labels correspond exactly to `decoder_input_ids`. 
    # We have to make sure that the PAD token is ignored
    batch["labels"] = [[-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch["labels"]]
    return batch

In [ ]:
batch_size = 6

In [ ]:
train_data = raw_ds.map(
    data_to_input, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["text"]
)

  0%|          | 0/210 [00:00<?, ?ba/s]

In [ ]:
# 모델이 쓰고 있는 torch의 텐서 형태로 정리함!
train_data.set_format(
    type="torch", columns=['input_ids', 'attention_mask', 'decoder_input_ids', 'decoder_attention_mask', 'labels'],
) 

In [ ]:
train_data = train_data.select(range(100, len(train_data)))

In [ ]:
val_data = train_data.select(range(100))

# Train

## Utils

## Implementation

In [ ]:
from transformers import Trainer, TrainingArguments

In [ ]:
# 엔지니어링! 몇번이나 백업할것인지, 내부 작업, beta값(Adam) ..
training_args = TrainingArguments(
    num_train_epochs=20,
    evaluation_strategy="steps",
    prediction_loss_only=True,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    # fp16=True, 
    # 저장할 디렉토리
    # 각자 자기 디렉토리 사용하기!!
    output_dir="./ckpt/YJ",

    # train loss 기록 주기
    logging_steps=100,

    # 파일 백업 저장 주기
    save_steps=500,

    # 평가주기
    eval_steps=100,

    learning_rate=8e-6,
    load_best_model_at_end=True,
    
    warmup_steps=500,
    save_total_limit=3,
)

In [ ]:
from torch import nn
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")

        loss_fct = nn.CrossEntropyLoss(reduction="mean")
        loss = loss_fct(logits.view(-1, self.model.config.decoder.vocab_size), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

In [ ]:
import torch

optims = torch.optim.SGD

In [ ]:
# instantiate trainer
trainer = CustomTrainer(
    model=bert2bert,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1155
  Num Epochs = 20
  Instantaneous batch size per device = 6
  Total train batch size (w. parallel, distributed & accumulation) = 6
  Gradient Accumulation steps = 1
  Total optimization steps = 3860
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.7/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:533: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss

Step,Training Loss,Validation Loss
100,3.541600,1.140352
200,1.433600,0.171743
300,0.471400,0.014938
400,0.186900,0.005071
500,0.093100,0.003056
600,0.053100,0.001674
700,0.027800,0.001616
800,0.022900,0.001320
900,0.014400,0.000724
1000,0.012600,0.000734


***** Running Evaluation *****
  Num examples = 100
  Batch size = 6
***** Running Evaluation *****
  Num examples = 100
  Batch size = 6
***** Running Evaluation *****
  Num examples = 100
  Batch size = 6
***** Running Evaluation *****
  Num examples = 100
  Batch size = 6
***** Running Evaluation *****
  Num examples = 100
  Batch size = 6
Saving model checkpoint to ./ckpt/YJ/checkpoint-500
Configuration saved in ./ckpt/YJ/checkpoint-500/config.json
Model weights saved in ./ckpt/YJ/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./ckpt/YJ/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./ckpt/YJ/checkpoint-500/special_tokens_map.json
/usr/local/lib/python3.7/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:533: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrep

TrainOutput(global_step=3860, training_loss=0.1535329830210289, metrics={'train_runtime': 1141.1502, 'train_samples_per_second': 20.243, 'train_steps_per_second': 3.383, 'total_flos': 858003384268800.0, 'train_loss': 0.1535329830210289, 'epoch': 20.0})